In [3]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(
    temperature=0.1
)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit= 120,
    return_messages=True,
)

examples = [
    {"input": "탑건", "output": "🛩️👨‍✈️🔥"},
    {"input": "대부", "output": "👨‍👨‍👦🔫🍝"},
    {"input": "타이타닉", "output": "🚢🧊❤️"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "영화 제목을 받으면, 그 영화를 나타내는 3개의 이모티콘으로 응답해야 하는 챗봇입니다."),
        few_shot_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}")
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm | StrOutputParser()

def invoke_chain(question):
    result = chain.invoke({"question":question})
    memory.save_context(
        {"input": question},
        {"output": result},
    )
    print(f"Human: {question}")
    print(f"AI: {result}")
    
invoke_chain("겨울왕국")
invoke_chain("반지의 제왕")


Human: 겨울왕국
AI: ❄️👸⛄️
Human: 반지의 제왕
AI: 🧝‍♂️💍👑


In [4]:
invoke_chain("처음으로 물어본 영화는?")

Human: 처음으로 물어본 영화는?
AI: 처음으로 물어본 영화는 '탑건'이었습니다!
